<a href="https://colab.research.google.com/github/makhmudovamunira/DataScience_Mohirdev/blob/main/Modelni_Baholash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### 5.1 - ML loyiha qadamlari

## Machine Learning

Nihoyat ma'lumotlarimiz ML uchun tayyor.

Boshlanishiga kerakli modullar va ma'lumotlarni qayta o'qib olamiz.

In [ ]:
import pandas as pd
import numpy as np
import sklearn

### Ma'lumotlarni o'qiymiz

In [ ]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set=train_test_split(df, test_size=0.2, random_state=42)
X_train=train_set.drop('median_house_value', axis=1)
y=train_set['median_house_value'].copy()

X_num=X_train.drop('ocean_proximity', axis=1)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedroom_ix, population_ix, household_ix=3,4,5,6

class ConbinedAttributesAdder(BaseEstimator, TransformerMixin):

  def __init__(self, add_bedroom_per_rooms=True):
    self.add_bedroom_per_rooms=add_bedroom_per_rooms
  def fit(self, X, y=None):
    return self

  def transform(self, X):
    rooms_per_household=X[:,rooms_ix]/X[:,household_ix]
    population_per_household=X[:,population_ix]/X[:,household_ix]

    if self.add_bedroom_per_rooms:
      bedrooms_per_room=X[:,bedroom_ix]/X[:,rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', ConbinedAttributesAdder(add_bedroom_per_rooms=True)),
    ('std_scaler', StandardScaler())
])

#### Matnli ustunlar uchun

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

Mana yakuniy, to'liq konveyer tayyor bo'ldi (`full_pipeline`).

Konveyerni ishga tushirish uchun `.fit_transform()` metodini chaqrisih kifoya.

In [ ]:
X_prepared=full_pipeline.fit_transform(X_train)

In [ ]:
X_prepared[:5, :]

Ma'lumotlar ML uchun tayyor.

### Machine Learning

Bizning maqsadimiz bashorat qilish, buning uchun bir nechta ML algoritmlar mavjud.

Biz kelgusi darslarda ularning har biri bilan yaqinda tanishamiz, hozir esa scikit-learn tarkibidagi ba'zi tayyor algoritmlardan foydalanamiz.

#### Linear Regression - Chiziqli regressiya
`sklearn` tarkibidagi `LinearRegression` klassidan yangi model yaratamiz.

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model=LinearRegression()

`LinearRegression` bu estimator. Estimatorlar ma'lumotlarni qabul qilib oladi va `.fit()` metodi yordamida ulardan basorat qilishni _o'rganadi_ (machine _learning_)

In [ ]:
LR_model.fit(X_prepared, y)

**TAMOM!** Machine Learning tugadi! Ha, adashmadingiz, bor yo'g'i 3 qator kod bilan biz hozirgina kompyuterga uylarni narxini bashorat qilishni o'rgatdik.

Modelni qanday qilib tekshirib ko'rishimiz mumkin? Keling `housing` datasetdan biror qatorni modelga beramiz va chiqqan natijani bizdagi bor natija (label) bilan solishtiramiz.

In [ ]:
test_data=X_train.sample(5)
test_data

In [ ]:
test_label=y.loc[test_data.index]
test_label

In [ ]:
test_data_prepared=full_pipeline.transform(test_data)
test_data_prepared

Predict(bashorat) qilamiz

In [ ]:
predict_data=LR_model.predict(test_data_prepared)
predict_data

In [ ]:
pd.DataFrame({'Prognoz': predict_data, 'Real baxosi': test_label})

### 5-QADAM. Modelni baholaymiz

Ko'rib turganingizdek, modelimiz qayerdadur kamroq, qayeradur ko'proq xato bilan bashorat qilgan.
Lekin model aniqligini baxolash uchun 5 qator yetarli emas. Keling, avvalroq ajratib olgan test set yordamida sinab ko'ramiz:

In [ ]:
test_set

In [ ]:
X_test=test_set.drop('median_house_value', axis=1)
X_test

In [ ]:
y_test=test_set['median_house_value'].copy()
y_test

In [ ]:
X_test_prepared=full_pipeline.transform(X_test)
y_predicted=LR_model.predict(X_test_prepared)

In [ ]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_test, y_predicted)
#RMSE ni hisoblaymi
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

In [ ]:
y_predicted.shape

Demak, `RMSE=72701$` chiqdi. Yomon emas, lekin yaxshi ham emas. Ya'ni modelimiz uylarni baholashda o'rtacha `72000$` ga adashayapti.

Model aniqligini oshirish uchun yagona, universal yechim yo'q. Qilib ko'rishingiz mumkin bo'lgan ishlar:
- Yaxhsiroq paramterlar topish
- Yaxhsiroq model (algoritm) tanlash
- Ko'proq ma'lumot yig'ish va hokazo.

Biz hozir boshqa model bilan sinab ko'ramiz.

### DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_model=DecisionTreeRegressor()
tree_model.fit(X_prepared, y)

In [ ]:
y_predicted=tree_model.predict(X_test_prepared)
y_predicted

In [ ]:
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

###Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model=RandomForestRegressor()
RF_model.fit(X_prepared, y)

In [ ]:
y_predicted=RF_model.predict(X_test_prepared)
lin_mse=mean_squared_error(y_test, y_predicted)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

## Cross-Validation usuli bilan baholash

Yuqorida biz modelni baholash uchun ma'lumotlarni test va train setlarga ajratdik.
Bu usulning kamchiligi biz test va train uchun doim bir xil ma'lumotlardan foydalanayapmiz.

Cross-validation yordamida biz ma'lumotlarni bir necha qismga ajratib, modelni turli qismlar yordamida bir nechta bor train va test qilishimiz mumkin.

Misol uchun, quyidagi rasmda ma'lumotlarni 5 ga ajratib train va test qilish ko'rsatilgan.

![](https://www.oreilly.com/library/view/machine-learning-quick/9781788830577/assets/b90b29ab-dfe7-4c11-9a2f-321e84f79495.png)

Cross validation uchun ma'lumotlarni train va testga bo'lish shart emas, buni sklearn o'zi qiladi.

In [ ]:
X=df.drop('median_house_value',axis=1)
y=df['median_house_value'].copy()

X_prepared=full_pipeline.transform(X)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

#### Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score


#### LogisticRegression

In [ ]:
scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

In [ ]:
display_scores(LR_rmse_scores)

In [ ]:
scores = cross_val_score(tree_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

In [ ]:
scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

## Modelni saqlash

Yaratgan modelimizdan kelajakda foydalanish uchun saqlab qo'yishimiz lozim. Umuman olganda nafaqat model, balki boshqa kerak bo'ladigan o'zgaruvchilarni ham saqlab qo'yish maqsadga muvvofiq bo'ladi. Masalan pipeline.

Buning uchun Pythondagi `pickle` yoki `joblib` modullaridan foydalanamiz.

### `pickle` yordamida saqlash

In [ ]:
import pickle

filename = 'RF_model.pkl'
with open(filename, 'wb') as file:
  pickle.dump(RF_model, file)

In [ ]:
with open(filename, 'rb') as file:
  model=pickle.load(file)

In [ ]:
scores = cross_val_score(model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

In [ ]:
model

### `joblib` yordamida saqlash
joblib katta NumPy martrisalarni siqib saqlash uchun afzal.

`joblib` o'rnatilmagan bo'lsa `pip install joblib` yordamida o'rnatib oling.

In [ ]:
import joblib

filename ='RF_model.jbl'
joblib.dump(RF_model, filename)

In [ ]:
model=joblib.load(filename)

In [ ]:
model

In [ ]:
scores = cross_val_score(model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

`pipeline` saqlab olamiz

In [ ]:
filename='pipeline.jbl'
joblib.dump(full_pipeline, filename)